In [1]:
import gensim, logging
# the model is organized like this: word = embeddings
model = gensim.models.KeyedVectors.load_word2vec_format('../resources/small-embeddings.txt', binary=False)


In [9]:
print (len(model["barack"]))

50


In [6]:
import nltk, string
from nltk.corpus import stopwords

exclude = set(string.punctuation)
stop_word_list = stopwords.words('english')

# input should be a string
def text_embedding(text):
    
    #it depends if the words have been lowercased or not
    text = text.lower()
    
    text = nltk.word_tokenize(text)
        
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    text = [token for token in text if token not in stop_word_list]
    print (text)
    article_embedd = []
    
    for word in text:
            try:
                embed_word = model[word]
                article_embedd.append(embed_word)
            except KeyError:
                continue
    print (len(article_embedd))
    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)]
    return avg


In [8]:
sentence = "Barack Obama was the president of the USA"

embed_sentence = text_embedding(sentence)
print (len(embed_sentence))

['barack', 'obama', 'president', 'usa']
4
50


In [10]:
# YELP product reviews dataset

import codecs

sentiment_dataset = codecs.open("../datasets/yelp-test.csv","r","utf-8").read().strip().split("\n")

print (sentiment_dataset[1])
print (" ")
print (sentiment_dataset[2])

"1","Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Never going back.  They seem overpriced, too."
 
"2","Friendly staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long."


In [11]:
# first, we define two folders, "corpus" - with the text and "labels", with the labels

corpus = []
labels = []

# be careful with this, the dataset is huge!
#for line in sentiment_dataset:
for line in sentiment_dataset[:1000]:
    text = line.split(",")[1].replace('"','')
    label = line.split(",")[0].replace('"','').replace("1","-1").replace("2","1")
    
    corpus.append(text)
    labels.append(label)

In [14]:
print (corpus[3])
print (labels[3])

The food is good. Unfortunately the service is very hit or miss. The main issue seems to be with the kitchen
-1


In [20]:
# AFINN Dictionary for Sentiment Analysis: https://github.com/fnielsen/afinn
#!pip install afinn

from afinn import Afinn

afinn = Afinn()

print (afinn.score("This is news"))



0.0


In [24]:
print (afinn.score("An exam in the mid of December? Oh, that's awesome!"))


4.0


In [25]:

print (afinn.score("That movie is horrible and beautiful at the same time"))


0.0


In [31]:
pred = []

for review in corpus:

    score = afinn.score(review)
    if score < 0.0:
        pred.append("-1")
    else:
        pred.append("1")

In [32]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print (precision_recall_fscore_support(labels, pred, average="macro"))

(0.7065650739683889, 0.6226429452570531, 0.5722449661809613, None)


Homework: Instead of taking the entire text, you can process it (e.g., remove the POS tagger, keep stopwords, etc) and see if you can improve the performance of the analysis

In [33]:
import numpy as np

X = np.array(corpus)
y = np.array(labels)

In [34]:
#here's the documentation: http://scikit-learn.org/stable/supervised_learning.html#supervised-learning
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB

final_f1 = []

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)

for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    classifier = GaussianNB().fit(X_train , y_train)
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
print (" ")
print (sum(final_f1)/len(final_f1))

/Users/federiconanni/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ValueError: Expected 2D array, got 1D array instead:
array=['Contrary to other reviews'
 'Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \\fixed\\ it for free'
 'Friendly staff'
 'The food is good. Unfortunately the service is very hit or miss. The main issue seems to be with the kitchen'
 "Picture Billy Joel's \\Piano Man\\ DOUBLED mixed with beer"
 'Mediocre service. COLD food! Our food waited so long the lettuce & pickles wilted. Bland food. Crazy overpriced. Long waits in the arcade. 1 beer per hour maximum.  Avoid at all costs. Fair manager.'
 'Ok! Let me tell you about my bad experience first. I went to D&B last night for a post wedding party - which'
 'I used to love D&B when it first opened in the Waterfront' 'Meh'
 'Unos has been around for ever'
 'Stars are for the food only.    2.5 rating\\n\\nGot there at 8pm on Saturday night'
 'Our last few visits h

In [ ]:
#here's the documentation: http://scikit-learn.org/stable/supervised_learning.html#supervised-learning
from sklearn import svm

SVM = svm.SVC(kernel = "linear", C=1) 

final_f1 = []

kf_total = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True)

for train, test in kf_total:
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    final_C = 1
    classifier = SVM.fit(X_train , y_train)
    y_pred = classifier.predict(X_test)
    
    print (precision_recall_fscore_support(y_test, y_pred, average="macro"))
    f1_score = precision_recall_fscore_support(y_test, y_pred, average="macro")[2]
    final_f1.append(f1_score)
print (" ")
print (sum(final_f1)/len(final_f1))